In [ ]:
## Reference: https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html

In [5]:
from io import open
import glob
import os
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker

def findFiles(path): return glob.glob(path)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Read a file and split into lines
def readLines(filename):
    with open(filename, encoding='utf-8') as some_file:
        return [unicodeToAscii(line.strip()) for line in some_file]

# Build the category_lines dictionary, a list of lines per category
category_lines = {}
all_categories = []
for filename in findFiles('../../data/data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

if n_categories == 0:
    raise RuntimeError('Data not found. Make sure that you downloaded data '
        'from https://download.pytorch.org/tutorial/data.zip and extract it to '
        'the current directory.')

print('# categories:', n_categories, all_categories)
print(unicodeToAscii("O'Néàl"))

# categories: 18 ['Russian', 'Polish', 'Greek', 'Spanish', 'Dutch', 'Czech', 'French', 'Korean', 'Arabic', 'Chinese', 'Vietnamese', 'Italian', 'Scottish', 'German', 'Portuguese', 'English', 'Japanese', 'Irish']
O'Neal


In [20]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size) 
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size) 
        self.o2o = nn.Linear(hidden_size + output_size, output_size) 
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)                     
        output = self.i2o(input_combined)                     
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)                   
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

print("n_letters: ",n_letters)
rnn = RNN(n_letters, 128, n_letters)

n_letters:  59


In [8]:
import random

# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line

c,l = randomTrainingPair()
print(c,l)

Arabic Bazzi


In [9]:
# One-hot vector for category
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# ``LongTensor`` of SECOND letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    
    return torch.LongTensor(letter_indexes)

In [18]:
line='abAc'
print([all_letters.find(line[li]) for li in range(1, len(line))])

[1, 26, 2]


In [24]:
# Make category, input, and target tensors from a random category, line pair
def randomTrainingExample():
    category, name     = randomTrainingPair() # eg: Arabic Bazzi
    category_tensor    = categoryTensor(category) # One-hot encoding for category
    input_name_tensor  = inputTensor(name) # One-hot encoding for name, mat: R*C, R: Num letters in the name. C: All letters' Len
    target_name_tensor = targetTensor(name) # Extract letter index from SECOND letter in the name, and append EOS on the tail. 
    return category_tensor, input_name_tensor, target_name_tensor

In [21]:
criterion = nn.NLLLoss()

learning_rate = 0.0005
print("n_letters: ",n_letters)
rnn = RNN(n_letters, 128, n_letters)

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = torch.Tensor([0]) # you can also just simply use ``loss = 0``

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item() / input_line_tensor.size(0)

n_letters:  59


In [22]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [25]:

n_iters = 100000
print_every = 5000
plot_every = 500
all_losses = []
total_loss = 0 # Reset every ``plot_every`` ``iters``

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

0m 13s (5000 5%) 3.6180
0m 27s (10000 10%) 3.1080
0m 43s (15000 15%) 2.8177
1m 0s (20000 20%) 2.5061
1m 15s (25000 25%) 1.9295
1m 31s (30000 30%) 2.9428
1m 48s (35000 35%) 2.5519
2m 5s (40000 40%) 2.8428
2m 24s (45000 45%) 2.3151
2m 41s (50000 50%) 2.5893
2m 58s (55000 55%) 2.0889
3m 14s (60000 60%) 3.3399
3m 31s (65000 65%) 3.2482
3m 48s (70000 70%) 2.7508
4m 4s (75000 75%) 1.9566
4m 20s (80000 80%) 3.4834
4m 37s (85000 85%) 2.1373
4m 53s (90000 90%) 2.2548
5m 9s (95000 95%) 2.7607
5m 25s (100000 100%) 1.8230


In [38]:
max_length = 20

# Sample from a category and starting letter
def sample(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(category, start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(category, start_letter))

# samples('Russian', 'RUS')

# samples('German', 'GER')

# samples('Spanish', 'SPA')

samples('Chinese', 'ZQSL')

Zhan
Quang
Shan
Lano
